# Table of Contents
 <p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Citation network of drafts of Chinese constitution

Hive Plot visualization

Next, try with graph_tool:

In [7]:
from graph_tool.all import *
import csv

hy = Graph()
v_name = hy.new_vertex_property("string")
v_color = hy.new_vertex_property("string")
v_size = hy.new_vertex_property("float")

# The file nodes.tsv contains the hierarchy of nodes, from a single "Origin" origin, in "from"-"to" relations
with open('nodes.tsv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter='\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        q  = row[0]       # from
        qc = row[1]       # from_color
        z  = row[2]       # to
        zc = row[3]       # to_color
        if len(graph_tool.util.find_vertex(hy, v_name, q)) == 0:
            s = hy.add_vertex()
            v_name[s] = q
            v_color[s] = qc
            if qc == "black": v_size[s] = 0
            else: v_size[s] = 10
        else:
            s = graph_tool.util.find_vertex(hy, v_name, q)[0]

        if len(graph_tool.util.find_vertex(hy, v_name, z)) == 0:
            t = hy.add_vertex()
            v_name[t] = z
            v_color[t] = zc
            if zc == "black": v_size[t] = 0
            else: v_size[t] = 10
        else:
            t = graph_tool.util.find_vertex(hy, v_name, z)[0]
        
        if not hy.edge(s, t):
            hy.add_edge(s, t)

pos = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

graph_draw(hy, pos=pos, vertex_text=v_name, vertex_fill_color=v_color, vertex_font_size=10, vertex_text_position="centered",
            output_size=(1024, 768), edge_pen_width=2, output="hierarchy.png")

if graph_tool.util.find_vertex(hy, "in", 0):
    l = graph_tool.util.find_vertex(hy, "in", 0)
    for n in l:
        print("Indegree problem with vertex {}".format(v_name[n]))

hy.save("hierarchy.xml.gz")

Indegree problem with vertex Origin


In [11]:
nw = Graph()
nwv_name  = nw.new_vertex_property("string")
nwv_color = nw.new_vertex_property("string")
nwv_size  = nw.new_vertex_property("float")
nwe_color = nw.new_edge_property("string")

for v in hy.vertices():
    n = nw.add_vertex()
    nwv_name[n]  = v_name[v]
    nwv_color[n] = v_color[v]
    nwv_size[n]  = v_size[v]

with open('edges_from_openrefine.tsv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter = '\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        source = ""
        target = ""
        
        f = row[0]       # from
        t = row[1]       # to
    
        if len(graph_tool.util.find_vertex(nw, nwv_name, f)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a source in edges list.".format(f))
            break
        else:
            source = graph_tool.util.find_vertex(nw, nwv_name, f)[0]
            c = nwv_color[source]
        
        if len(graph_tool.util.find_vertex(nw, nwv_name, t)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a target in edges list.".format(t))
            break
        else:
            target = graph_tool.util.find_vertex(nw, nwv_name, t)[0]
        
        e = nw.add_edge(source, target)
        nwe_color[e] = c
        print("Create " + c + " edge from '" + f + "' to '" + t + "'.")

nw.save("network.xml.gz")

graph_draw(nw, vertex_text=nwv_name, vertex_fill_color=nwv_color, vertex_font_size=10, vertex_text_position=10,
           output_size=(1024, 768), edge_pen_width=2, output="network_1.png")


Create limegreen edge from 'Draft 1923_1923/1' to 'Germany 1919_Art.151'.
Create limegreen edge from 'Draft 1923_1923/1' to 'Germany 1919'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Pre-Qin Scholars'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Book of Mencius'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919_Art.155 (1)'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919_Art.153'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Constitutions of Modern Nations'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Germany 1919'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'The Soviet Union 1936'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Poland 1921'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Poland 1921'.
Create limegreen edge from 'Draft 1923_1923/2 (1)' to 'Portugal 1911'.
Create limegreen edge from 'Draft 19

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f0ebf921208, at 0x7f0ec0adbeb8>

In [12]:
# state = graph_tool.inference.minimize.minimize_nested_blockmodel_dl(nw, deg_corr=True)
# t     = graph_tool.inference.nested_blockmodel.get_hierarchy_tree(state)[0]

hypos  = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

cts   = graph_tool.draw.get_hierarchy_control_points(nw, hy, hypos, beta=0.8, max_depth=3)
pos   = nw.own_property(hypos)

graph_draw(nw, pos=pos, vertex_text=nwv_name, vertex_fill_color=nwv_color,
           vertex_font_size=nwv_size, vertex_size=nwv_size, vertex_text_position="centered",
           edge_control_points=cts, edge_pen_width=1, edge_color=nwe_color,
           output_size=(1500, 1500), output="network_2.png")


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f0ebf921208, at 0x7f0ef8300668>